In [1]:
using LinearAlgebra
using SparseArrays
using Arpack
using KrylovKit
using Dates
using JLD

## Specifying parameters of the model here:

In [2]:
L=4;J=1;h=1000

1000

## Building up Hamiltonian without using k space
- Building Hamiltonian for the simplest square lattice heisenberg model
- Using Int64 to represent states
- start by a 2 by 2 lattice, so takes 4 bits out of the 64 bits of an Int64.

Hamiltonian is $H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $

First find all neighbours of a given site in the binary representation. then build up a matrix of 16 by 16
The k-th bit of a binary number correspond to the (i,j) site on the lattice, for a lattice of linear size L. Which could be calculated by this following function:

In [3]:
function coordinate(n;L1::Int64=2,L1::Int64=2)
    num_sites = L^2
    i::Int64 = Int(ceil(n/L))
    j::Int64 = mod1(n,L)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};L::Int64=2)
    n = (coordinate[1]-1)*L + coordinate[2]
    return n
end# Now Let's Construct Hamiltonian for our problem, by changing neighbor lists

bit_pos (generic function with 1 method)

# Now Let's Construct Hamiltonian for our problem, by changing neighbor lists

In [4]:
function neib(n::Int64;L::Int64=2)
    coord = coordinate(n,L=L)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L)))    
    if iseven(coord[1]+coord[2])
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]-1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]+1,L)))
    else
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]+1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]-1,L)))
    end
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib,L=L))
    end
    return neibs_bit_pos
end

function neib_list_gen(;L::Int64=L)
    neib_list = Set{Int64}[]
    for n in 1:L^2
        push!(neib_list, neib(n, L=L))
    end
    return neib_list
end
neib_list = neib_list_gen(L=L)

16-element Array{Set{Int64},1}:
 Set([4, 13, 14, 2, 5, 8])  
 Set([7, 14, 13, 3, 6, 1])  
 Set([7, 4, 2, 16, 15, 6])  
 Set([3, 16, 8, 5, 15, 1])  
 Set([9, 4, 10, 8, 6, 1])   
 Set([7, 9, 10, 2, 3, 5])   
 Set([2, 3, 11, 8, 6, 12])  
 Set([7, 4, 11, 5, 12, 1])  
 Set([13, 10, 16, 5, 12, 6])
 Set([9, 14, 11, 5, 15, 6]) 
 Set([7, 14, 10, 8, 15, 12])
 Set([7, 9, 13, 16, 11, 8]) 
 Set([9, 14, 2, 16, 12, 1]) 
 Set([13, 10, 2, 11, 15, 1])
 Set([4, 14, 10, 3, 16, 11])
 Set([4, 13, 9, 3, 15, 12]) 

# Now we can construct Hamlitonian for our system
## Let's **NOT** make use of the Parity Symmetry for now

Our Hamiltonian is originally 

\begin{equation}
\tag{H1}
H=\sum_{i=1}^{N}\sum_{\sigma}J S_i^z S_{i+\sigma}^z + h\sum_{i=1}^{N} S_i^x
\end{equation}

Or equivalently, we can do a little rotation of our system and construct Hamiltonian in that "direction". The Hamiltonian should be:

\begin{equation}
\tag{H2}
H=\sum_{i=1}^{N}\sum_{\sigma}J S_i^x S_{i+\sigma}^x - h\sum_{i=1}^{N} S_i^z
\end{equation}

To simplify the meansurement of operators, we could make use of both the two coordinate systems. Since we're working in the z basis in either cases, the first(H1) comes in handy for measuring spins along the coupled direction(perpendicular to h), the second one comes in handy in measuring spin alone the direction of the field h.

Equivalently, we can write our Hamiltonian as 

\begin{equation}
\tag{H1}
H=\sum_{i=1}^{N}\sum_{\sigma}JS_i^z S_{i+\sigma}^z + \frac{h}{2}\sum_{i=1}^{N} (S_i^+ + S_i^-)
\end{equation}

\begin{equation}
\tag{H2}
H=\sum_{i=1}^{N}\sum_{\sigma}\frac{J}{4}(S_i^+ + S_i^-)(S_{i+\sigma}^+ + S_{i+\sigma}^-) - h\sum_{i=1}^{N} S_i^z
\end{equation}


## Generating Hamiltonian H1 and H2

First thing first, we need to generate Hamiltonian H1 and H2 with the following 2 functions:

In [5]:
function update_val(row_inds, col_inds, vals;row_ind, col_ind, val)
    push!(row_inds, row_ind)
    push!(col_inds, col_ind)
    push!(vals, val)
end

#=function Hamiltonian1(;L::Int64=2, J=1, h=1, neib_list)
    H = spzeros(2^(L^2),2^(L^2))
    for state in 0:(2^(L^2)-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            flipped_state = state ⊻ (1<<(i-1))
            H[state+1,flipped_state+1] += (1/2)*h
            for j in neib_list[i] #loop over(compare) all neighbors of a given site
                H[state+1,state+1] += (state_binary[i]-1/2)*(state_binary[j]-1/2)*J/2   
            end
        end
    end
    return H
end=#

function Hamiltonian1(;L::Int64=2, J=1, h=1, neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^(L^2)-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            flipped_state = state ⊻ (1<<(i-1))
            update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val = (1/2)*h)
            for j in neib_list[i] #loop over(compare) all neighbors of a given site
                update_val(row_inds, col_inds, vals, row_ind = state+1, col_ind = state+1, val = (state_binary[i]-1/2)*(state_binary[j]-1/2)*J/2)
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^(L^2), 2^(L^2), +)
end

function Hamiltonian2(;L::Int64=2, J=1, h=1, neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^(L^2)-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            if state_binary[i] == 1
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val = - h/2)
            else
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val =  h/2)
            end
            for j in neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val =  (1/4)*(J/2))
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^(L^2), 2^(L^2), +)
end

#=function Hamiltonian2(;L::Int64=2, J=1, h=1, neib_list)
    H = spzeros(2^(L^2),2^(L^2))
    for state in 0:(2^(L^2)-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            if state_binary[i] == 1
                H[state+1,state+1] -= h/2 
            else
                H[state+1,state+1] += h/2
            end
            for j in neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                H[state+1,flipped_state+1] += (1/4)*(J/2)
            end
        end
    end
    return H
end=#

@time H1 = Hamiltonian1(;L=L, J=J, h=h, neib_list=neib_list)
@time H2 = Hamiltonian2(;L=L, J=J, h=h, neib_list=neib_list)

  0.594657 seconds (770.90 k allocations: 399.900 MiB, 15.82% gc time)
  0.407441 seconds (151.32 k allocations: 400.954 MiB, 13.86% gc time)


65536×65536 SparseMatrixCSC{Float64,Int64} with 3211264 stored entries:
  [1    ,     1]  =  8000.0
  [4    ,     1]  =  0.25
  [7    ,     1]  =  0.25
  [10   ,     1]  =  0.25
  [13   ,     1]  =  0.25
  [18   ,     1]  =  0.25
  [25   ,     1]  =  0.25
  [35   ,     1]  =  0.25
  [37   ,     1]  =  0.25
  [49   ,     1]  =  0.25
  [67   ,     1]  =  0.25
  [69   ,     1]  =  0.25
  ⋮
  [65440, 65536]  =  0.25
  [65468, 65536]  =  0.25
  [65470, 65536]  =  0.25
  [65488, 65536]  =  0.25
  [65500, 65536]  =  0.25
  [65502, 65536]  =  0.25
  [65512, 65536]  =  0.25
  [65519, 65536]  =  0.25
  [65524, 65536]  =  0.25
  [65527, 65536]  =  0.25
  [65530, 65536]  =  0.25
  [65533, 65536]  =  0.25
  [65536, 65536]  =  -8000.0

## Using Lanczos to calculate the ground state and compare see if the Hamiltonians are compatible

I'm using ```KrylovKit.jl``` to calculate.

It appears that the eigenstates is automatically normalized by the package.

In [6]:
eigs1 = eigsolve(H1, 1, :SR, eltype(H1), tol = 10^(-20))
eigs2 = eigsolve(H2, 1, :SR, eltype(H2), tol = 10^(-20))
eigstate1 = eigs1[2][1]./(conj.(eigs1[2][1]')*eigs1[2][1])
eigstate2 = eigs2[2][1]./(conj.(eigs2[2][1]')*eigs2[2][1])

65536-element Array{Float64,1}:
  1.3039055520632788e-16
  2.31772215000641e-17  
  3.915311972376871e-17 
 -6.32542793915564e-18  
  3.534228904715122e-18 
 -1.6818345464609513e-17
 -2.156044812553797e-18 
  2.019324749982158e-17 
  5.418605936034917e-19 
 -6.2672679967583024e-18
 -1.5248159515710016e-17
  1.8546402459688926e-17
 -6.420571507563613e-19 
  ⋮                     
  1.9744609013725405e-17
  6.23829758502004e-8   
 -0.00012493751946892422
 -2.686145312753168e-16 
  9.799564303652452e-18 
 -0.00012493751946892497
  6.238297619134349e-8  
 -3.659868290631243e-19 
 -0.00012493751946942574
 -1.3613503501460194e-15
 -1.7206281867703562e-15
  0.9999996253744571    

## In case we need to normalize the eigenstates
```julia
eigstate1 = eigs1[2][1]./(conj.(eigs1[2][1]')*eigs1[2][1])
eigstate2 = eigs2[2][1]./(conj.(eigs2[2][1]')*eigs2[2][1])
```

# Now we could calculate quantities given that we have the ground state.

Here x means the direction of h, y means the direction of coupling.

- $m=\frac{1}{N}\sum_{i} \langle \sigma_i^x \rangle$

Calculating this using H2

- $S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

Calculating this using H1

- $F_{cl}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle f_{p_1} f_{p_2}\rangle$

Calculating this using H1

- $F_{QM}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle o_{p_1} o_{p_2}\rangle$ with $o_p = \sigma_1^+\sigma_2^-\sigma_3^+\sigma_4^- + \sigma_1^-\sigma_2^+\sigma_3^-\sigma_4^+$

Calculating this using H1? I think

- $S_A = -Tr \rho_A \log{\rho_A}$

Where $\rho_A = Tr_B \rho$

Calculating this using either

- $Fidelity = 2\cdot\frac{1 - |\langle \psi_0(h)|\psi_0(h+\delta h)\rangle|}{\delta h^2} $

Calculating this using either




## Measuring m using H2

We basically just need to construct matrix of $m$ in the basis out which you constructed the hamiltonian H2.

In H2, $m$ is the magnetization along z direction. So:

$m=\frac{1}{N}\sum_{i} \langle \sigma_i^z \rangle$

In [7]:
function m_H2(state::Array{Float64,1}; L::Int64)
    m = spzeros(2^(L^2),2^(L^2))
    for basis_state in 0:(2^(L^2)-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        # calculating total spin along z direction, considering it's spin 1/2
        m[basis_state+1, basis_state+1] += (sum(basis_state_binary)-1/2*L^2)/L^2
    end
    #now that we have matrix m, calculate the average m_val:
    m_val = conj.(state')*m*state
    return m_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

m_H2(eigstate2; L=L)

0.4999999063436016

## Measuring $S_{\pi}$ using H1

Constructing $S_{\pi}$ matrix using basis of $H1$

$S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

In [8]:
function S_pi_H1(state::Array{Float64,1}; L::Int64, neib_list)
    S_pi = spzeros(2^(L^2),2^(L^2))
    for basis_state in 0:(2^(L^2)-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            for j in 1:L^2 #loop over all sites again
                 S_pi[basis_state+1,basis_state+1] += (basis_state_binary[i]-1/2)*(basis_state_binary[j]-1/2)*(-1)^(sum(coordinate(i;L=L))+sum(coordinate(j;L=L)))/(L^2)
            end
        end
    end
    #now that we have matrix m, calculate the average m_val:
    S_pi_val = conj.(state')*S_pi*state
    return S_pi_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

function S_pi_H1_abs(state::Array{Float64,1}; L::Int64, neib_list)
    S_pi = spzeros(2^(L^2),2^(L^2))
    for basis_state in 0:(2^(L^2)-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        for i in 1:L^2 #loop over all sites in a given state
            for j in 1:L^2 #loop over all sites again
                 S_pi[basis_state+1,basis_state+1] += abs((basis_state_binary[i]-1/2)*(basis_state_binary[j]-1/2))/(L^2)
            end
        end
    end
    #now that we have matrix m, calculate the average m_val:
    S_pi_val = conj.(state')*S_pi*state
    return S_pi_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

S_pi_H1(eigstate1; L=L, neib_list=neib_list)

0.2501249531405267

## Measuring staggered flippability

We first need to find all the plaquettes . Generate a list of plaquettes.

### Classical Staggered Flippability

$F_{cl}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle f_{p_1} f_{p_2} \rangle (-1)^{p_1+p_2}$

### Quamtum Staggered Flippability

$F_{QM}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle o_{p_1} o_{p_2} \rangle (-1)^{p_1+p_2}$ with $o_p = \sigma_1^+\sigma_2^-\sigma_3^+\sigma_4^- + \sigma_1^-\sigma_2^+\sigma_3^-\sigma_4^+$

In [9]:
function plaquette_list_gen(;L, neib_list)
    plaquette_list = []
    for site in 1:L^2 #loop over all sites
        site_coord = coordinate(site,L=L)
        if bit_pos((mod1(site_coord[1]-1, L),mod1(site_coord[2]-1, L)),L=L) ∈ neib_list[site]
            plaq_site = Int64[] #Has to be an array to make it ordered
            push!(plaq_site, site)
            push!(plaq_site, bit_pos((site_coord[1],mod1(site_coord[2]-1, L)),L=L))
            push!(plaq_site, bit_pos((mod1(site_coord[1]+1, L),mod1(site_coord[2]-1, L)),L=L))
            push!(plaq_site, bit_pos((mod1(site_coord[1]+1, L),site_coord[2]),L=L))
            #push the found plaquette to the plaquette list
            push!(plaquette_list, plaq_site)
        end
    end
    return plaquette_list
end

plaquette_list = plaquette_list_gen(;L=L, neib_list=neib_list)
println(plaquette_list)

function plaquette_phase_factors_gen(;L::Int64, plaquette_list)
    @assert(iseven(L), "L must be even for our lattice")
    phase_factors = Int64[]
    for p in 1:Int(L/2)
        for j in 1:Int(L/2)
            push!(phase_factors, 0)
        end
        for k in 1:Int(L/2)
            push!(phase_factors, 1)
        end
    end
    return phase_factors
end
plaquette_phase_factors = plaquette_phase_factors_gen(;L=L, plaquette_list = plaquette_list)
println(plaquette_phase_factors)

function is_flippable(state_binary::Array, plaquette_sites::Array{Int64, 1} )
    spin_plaquette = Int64[]
    for site in plaquette_sites
        push!(spin_plaquette, state_binary[site])
    end
    if (spin_plaquette - [1;0;1;0] == [0;0;0;0]) || (spin_plaquette - [0;1;0;1] == [0;0;0;0])
        return true
    else
        return false
    end
end    

println(is_flippable(digits!(zeros(Int64, 64), 10, base = 2), [1,2,3,4]))
    
function Fcl(state::Array{Float64,1}; L::Int64, neib_list, plaquette_list)
    Fcl = spzeros(2^(L^2),2^(L^2))
    for basis_state in 0:(2^(L^2)-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        plaquette_list_len = length(plaquette_list)
        for i in 1:plaquette_list_len                
            for j in 1:plaquette_list_len
                if is_flippable(basis_state_binary, plaquette_list[i]) && is_flippable(basis_state_binary, plaquette_list[j])
                    Fcl[basis_state+1, basis_state+1] += (-1)^(plaquette_phase_factors[i]+plaquette_phase_factors[j]) / plaquette_list_len
                end
            end
        end
    end
    # calculating Fcl value for the state
    Fcl_val = conj.(state')*Fcl*state
    return Fcl_val[1]
end

println("Fcl of state is "* "$(Fcl(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))")

function Fqm(state::Array{Float64,1}; L::Int64, neib_list, plaquette_list)
    Fqm = spzeros(2^(L^2),2^(L^2))
    for basis_state in 0:(2^(L^2)-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        plaquette_list_len = length(plaquette_list)
        for i in 1:plaquette_list_len                
            for j in 1:plaquette_list_len
                if is_flippable(basis_state_binary, plaquette_list[i]) && is_flippable(basis_state_binary, plaquette_list[j])
                    flipped_state = basis_state
                    for site in vcat(plaquette_list[i], plaquette_list[j])
                        flipped_state = flipped_state ⊻ (1<<(site-1))
                    end
                    Fqm[basis_state+1, flipped_state+1] += (-1)^(plaquette_phase_factors[i]+plaquette_phase_factors[j]) / plaquette_list_len
                end
            end
        end
    end
    # calculating Fcl value for the state
    Fqm_val = conj.(state')*Fqm*state
    return Fqm_val[1]
end

println("Fqm of state is "* "$(Fqm(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))")

Any[[2, 1, 5, 6], [4, 3, 7, 8], [5, 8, 12, 9], [7, 6, 10, 11], [10, 9, 13, 14], [12, 11, 15, 16], [13, 16, 4, 1], [15, 14, 2, 3]]
[0, 0, 1, 1, 0, 0, 1, 1]
true
Fcl of state is 0.10949999999407957
Fqm of state is 0.1095625546386067


## Measuring entanglement entropy

We need to rearrange the coefficients(in another word, wave function) of the ground state to a matrix. With the rows of the matrix represent states of the sub system, columns of the matrix represent states of the environment.

To put it explicitly, originally we write every state in the system as 

\begin{equation}
\tag{original}
|GS\rangle = \sum_{i} C_i |\psi_i\rangle
\end{equation}

We separate out a small part (part A) of the system, and treat the rest (part B) of the system as environment. 

\begin{equation}
\tag{part A seperated}
|GS\rangle = \sum_{i} C_{a,b} |\psi_a\rangle \otimes |\psi_b\rangle
\end{equation}

Then we could do a SVD decomposition of the coefficient matrix $C_{a,b}$, 

\begin{equation}
\tag{SVD}
C = U^+ D V
\end{equation}

With D being a diagonal matrix. The diagonal matrices are "singular values" $\lambda_i$. 

The entanglement Entropy can be calculated as:

\begin{equation}
\tag{Entanglement Entropy}
S = -\sum_i \lambda_i^2 \log{\lambda_i^2}
\end{equation}


### Setting up correspondance between a state number i and the a,b number pair

$a$ is the number the state of the sub system corresponds to. $b$ is the number the state of the environment corresponds to.

As an example: suppose A is site 1,2; environment is site 3-8:

state $|11;110000 \rangle$ corresponds to $i=15$ and also $a=3,b=3$

In [10]:
function S_entangle(state::Array{Float64,1}; L::Int64, neib_list)
    #find the first four spin interacting plaquette
    plaq_sites = Int64[] #Has to be an array to make it ordered
    for site in 1:L^2 #loop over all sites
        site_coord = coordinate(site,L=L)
        if bit_pos((mod1(site_coord[1]-1, L),mod1(site_coord[2]-1, L)),L=L) ∈ neib_list[site]
            push!(plaq_sites, site)
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L),site_coord[2]),L=L))
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L),mod1(site_coord[2]+1, L)),L=L))
            push!(plaq_sites, bit_pos((site_coord[1],mod1(site_coord[2]+1, L)),L=L))
            break
        end
    end
    #println(plaq_sites)
    env_sites = Int64[]
    for site in 1:L^2
        if site ∉ plaq_sites
            push!(env_sites, site)
        end
    end
    #println(env_sites)
    C = zeros(Number, 16, Int(2^(L^2)/16))
    for basis_state in 0:(2^(L^2) - 1)
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        a = 1; b = 1 # start at 1 to avoid 0 as numeration number
        for i in 1:4
            a += basis_state_binary[plaq_sites[i]] * 2^(i-1)
        end
        for i in 1:(L^2-4)
            b += basis_state_binary[env_sites[i]] * 2^(i-1)
        end
        C[a, b] = state[basis_state + 1]        
    end
    Sing_vals = svd(C).S
    return  -sum(Sing_vals.^2 .* log.(Sing_vals.^2))
end

S_entangle(eigstate1; L=L, neib_list=neib_list)

3.3485504787927675e-6

In [11]:
a=[1;0]
b=[1;0]
c = vcat(a,b)
#sz = conj.(a')*b*a
typeof(c)
#println(c)
Int(0b11)

3

## Below we write a driver  to calculate m and $S_{\pi}$ for different h

Calculate Fidelity in the driver

In [16]:
function driver(;L=L, h_vals)
    J = 1
    neib_list = neib_list_gen(L=L)
    plaquette_list = plaquette_list_gen(;L=L, neib_list=neib_list)
    m = []
    S_pi = []
    Fidelity = []
    Fcl_vals = []
    Fqm_vals = []
    S_entangle_vals = []
    eigstate_prev = zeros(Float64, 2^(L^2))
    h_prev = -1
    for i in 1:length(h_vals)
        h = h_vals[i]
        H1 = Hamiltonian1(;L=L, J=J, h=h, neib_list=neib_list)
        H2 = Hamiltonian2(;L=L, J=J, h=h, neib_list=neib_list)
        eigstate1 = eigsolve(H1, 1, :SR, eltype(H1), tol = 10^(-20))[2][1]
        eigstate2 = eigsolve(H2, 1, :SR, eltype(H2), tol = 10^(-20))[2][1]
        #calculating Fidelity using eigenstates of H1
        Fid = conj.(eigstate_prev')*eigstate1
        (i == 1) && (Fid[1] = 1) #set Fid to be 1 manually for the first h
        push!(m, m_H2(eigstate2; L=L))
        push!(S_pi, S_pi_H1(eigstate1; L=L, neib_list=neib_list))
        push!(Fidelity, 2*(1-abs(Fid[1]))/(h-h_prev)^2)
        push!(Fcl_vals, Fcl(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        push!(Fqm_vals, Fqm(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        push!(S_entangle_vals, S_entangle(eigstate1; L=L, neib_list=neib_list))
        eigstate_prev = eigstate1
        h_prev = h
    end
    return (h_vals, m, S_pi, Fidelity, Fcl_vals, Fqm_vals, S_entangle_vals)
end

L=4; h_vals = 1 ./ ((range(0.1, 0.0001, length = 100)).^2)
@time result = driver(L=L, h_vals=h_vals)

my_time = Dates.now()

time_finished = "Date_$(Dates.format(my_time, "e_dd_u_yyyy_HH_MM_SS"))"
content = "Square_Spin_Ice_Measurement"
save_path = "E:/UC Davis/Research/Square Spin Ice/Square-Spin-Ice/Yutan_code/Results/"
#"/nfs/home/zyt329/Research/Square_spin_ice/result/"
save_name = save_path*content*"_L=$(L)_hmin=$(h_vals[1])_hmax=$(h_vals[end])_"*time_finished*".jld"

save(save_name, "result", result)
println("finished")

794.139285 seconds (7.65 G allocations: 798.580 GiB, 13.13% gc time)
finished


In [33]:
log(6)

1.791759469228055

In [33]:
minimum(eigs2[1])

-9.039382249249678

## check S_entangle code sith 4 sites

Check with the state $1/\sqrt{2} (|1111\rangle + |0000\rangle)$

In [28]:
function S_entangle(state::Array{Float64,1}; L::Int64, neib_list)
    #println(plaq_sites)
    plaq_sites = [1]
    env_sites = Int64[]
    for site in 1:L^2
        if site != 1
            push!(env_sites, site)
        end
    end
    #println(env_sites)
    C = zeros(Number, 2, Int(2^(L^2)/2))
    for basis_state in 0:(2^(L^2) - 1)
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        a = 1; b = 1 # start at 1 to avoid 0 as numeration number
        for i in 1
            a += basis_state_binary[plaq_sites[i]] * 2^(i-1)
        end
        for i in 1:(L^2-1)
            b += basis_state_binary[env_sites[i]] * 2^(i-1)
        end
        C[a, b] = state[basis_state + 1]        
    end
    Sing_vals = svd(C).S
    return  -sum(Sing_vals.^2 .* log.(Sing_vals.^2))
end

S_entangle([0.7071067811865475,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.7071067811865475]; L=2, neib_list=neib_list)

0.6931471805599454

In [17]:
eigen(H1)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
16-element Array{Float64,1}:
 -0.5
 -0.5
 -0.5
 -0.5
 -0.5
 -0.5
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  1.5
  1.5
eigenvectors:
16×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0

In [23]:
(row_inds, col_inds, vals) = ([],[],[])
function update_val(row_inds, col_inds, vals;row_ind, col_ind, val)
    push!(row_inds, row_ind)
    push!(col_inds, col_ind)
    push!(vals, val)
end

update_val(row_inds, col_inds, vals, row_ind=1, col_ind=1, val=1)
update_val(row_inds, col_inds, vals, row_ind=2, col_ind=2, val=2)
vals

2-element Array{Any,1}:
 1
 2